# Paso 1: Configuración del Entorno (Google Colab)

**1.1 Importación de Librerías**

In [68]:
# BITÁCORA: PASO 1.1 - Importación de Librerías Necesarias
# Pandas: Para manejar DataFrames (tablas).
# Scikit-learn: Librería principal para Machine Learning (IA).

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from IPython.display import display
from google.colab import drive # Necesario para montar tu Drive

**1.2 Montar Google Drive**

In [12]:
# BITÁCORA: PASO 1.2  - Montar Google Drive
# Ejecuta esta celda y sigue el enlace para dar permiso a Colab de acceder a tu Drive.
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive montado con éxito. Puedes encontrar tus archivos en la carpeta '/content/drive/MyDrive'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado con éxito. Puedes encontrar tus archivos en la carpeta '/content/drive/MyDrive'.


# Paso 2: Carga y Exploración del Archivo de Muestra

**2.1 Carga del Dataset**

In [17]:
# BITÁCORA: PASO 2.1 - Carga del Dataset de Muestra (¡Con SINTAXIS CORRECTA!)
import pandas as pd # Necesario para leer el CSV

# ¡CORRECCIÓN CLAVE! LA RUTA DEBE ESTAR ENTRE COMILLAS SIMPLES O DOBLES
ruta_csv = '/content/drive/MyDrive/Colab Notebooks/DataAnalytics/iotsim-hydraulic-system-2.csv'
df = pd.read_csv(ruta_csv)

print("✅ Carga de datos exitosa. La tabla 'df' ha sido creada.")

✅ Carga de datos exitosa. La tabla 'df' ha sido creada.


**2.2 Exploración y Verificación de Columnas:**

Debemos verificar si el archivo es el correcto y cómo se llaman las columnas de Ataque y Protocolo.

In [67]:
# BITÁCORA: PASO 2.2 - Exploración Final (Verificación)

print("Primeras 5 filas del dataset:")
display(df.head()) # Muestra la tabla (DataFrame)

print("\nConteo de valores en la columna de ataque/etiqueta ('label'):")
display(df['label'].value_counts()) # Confirma los ataques

Primeras 5 filas del dataset:


,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,label
0,"Jan 14, 2025 18:40:10.397697000 GMT",68,eth:ethertype:ip:tcp:mqtt,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,6,...,1883.0,0x0018,502.0,-1.0,0x1fd0,0101080a6be0130a27064a75,2.0,NaN,NaN,Benign
1,"Jan 14, 2025 18:40:10.405220000 GMT",68,eth:ethertype:ip:tcp:mqtt,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.4.2,0x02,61,6,...,44229.0,0x0018,501.0,-1.0,0xae0f,0101080a2706ac346be0130a,2.0,NaN,NaN,Benign
2,"Jan 14, 2025 18:40:10.405272000 GMT",66,eth:ethertype:ip:tcp,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,6,...,1883.0,0x0010,502.0,-1.0,0x7e10,0101080a6be013112706ac34,NaN,NaN,NaN,Benign
3,"Jan 14, 2025 18:40:40.388553000 GMT",67,eth:ethertype:ip:udp:dns,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.0.2,192.168.20.31,0x02,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40294.0,53.0,Benign
4,"Jan 14, 2025 18:40:40.405604000 GMT",83,eth:ethertype:ip:udp:dns,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.0.2,0x02,61,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,40294.0,Benign



Conteo de valores en la columna de ataque/etiqueta ('label'):


,count
label,
Benign,2501


# Paso 3: Pre-procesamiento y Ingeniería de Características

Transformamos las palabras (etiquetas de ataque) en números (0 o 1) y preparamos las demás variables.


**Listado de Columnas:**

revisar si los nombres de las columnas

In [22]:
# BITÁCORA paso 3.0 - ver Listado de Columnas (¡Busca el nombre exacto de la etiqueta!)

print("Lista de nombres de todas las columnas en tu archivo:")
# ¡Este es el comando clave!
print(df.columns.tolist())

Lista de nombres de todas las columnas en tu archivo:
['frame.time', 'frame.len', 'frame.protocols', 'eth.src', 'eth.dst', 'ip.dst', 'ip.src', 'ip.flags', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.tos', 'tcp.srcport', 'tcp.dstport', 'tcp.flags', 'tcp.window_size_value', 'tcp.window_size_scalefactor', 'tcp.checksum', 'tcp.options', 'tcp.pdu.size', 'udp.srcport', 'udp.dstport', 'label']


**3.1 Definición de Features (X) y Target (y):**

revisar si los nombres de las columnas (pkt_len_max, protocol_type, etc.) coinciden

In [51]:
# BITÁCORA: PASO 3.1 - Definición de Features (X) y Target (y)
# 'X' contendrá las columnas que usaremos para predecir (todas menos 'label').
# 'y' contendrá la columna que queremos predecir ('label').

# Verifica que la columna 'label' exista antes de usarla
if 'label' in df.columns:
    X = df.drop('label', axis=1) # Todas las columnas EXCEPTO 'label' son features
    y = df['label']             # 'label' es la variable objetivo (target)

    print("✅ Variables X (features) y y (target) definidas con éxito.")
    print("\nPrimeras 5 filas de X (features):")
    display(X.head())
    print("\nPrimeras 5 filas de y (target):")
    display(y.head())
else:
    print("❌ Error: La columna 'label' no se encontró en el DataFrame. Por favor, verifica el nombre de la columna.")


✅ Variables X (features) y y (target) definidas con éxito.

Primeras 5 filas de X (features):


,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.srcport,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport
0,"Jan 14, 2025 18:40:10.397697000 GMT",68,eth:ethertype:ip:tcp:mqtt,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,6,...,44229.0,1883.0,0x0018,502.0,-1.0,0x1fd0,0101080a6be0130a27064a75,2.0,NaN,NaN
1,"Jan 14, 2025 18:40:10.405220000 GMT",68,eth:ethertype:ip:tcp:mqtt,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.4.2,0x02,61,6,...,1883.0,44229.0,0x0018,501.0,-1.0,0xae0f,0101080a2706ac346be0130a,2.0,NaN,NaN
2,"Jan 14, 2025 18:40:10.405272000 GMT",66,eth:ethertype:ip:tcp,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,6,...,44229.0,1883.0,0x0010,502.0,-1.0,0x7e10,0101080a6be013112706ac34,NaN,NaN,NaN
3,"Jan 14, 2025 18:40:40.388553000 GMT",67,eth:ethertype:ip:udp:dns,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.0.2,192.168.20.31,0x02,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40294.0,53.0
4,"Jan 14, 2025 18:40:40.405604000 GMT",83,eth:ethertype:ip:udp:dns,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.0.2,0x02,61,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,40294.0



Primeras 5 filas de y (target):


,label
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign


**3.2 Codificación de Características Categóricas (One-Hot Encoding):**


Convertimos la columna protocol_type (ej. 'TCP', 'UDP') a números que el modelo pueda procesar.

In [52]:
# BITÁCORA: PASO 3.2 - Codificación Categórica (One-Hot Encoding)
# Convierte el protocolo (ej. 6 para TCP, 17 para UDP) a columnas binarias.
X = pd.get_dummies(X, columns=['ip.proto'])

print("✅ 2. Características después de la Codificación (Protocolos convertidos).")
print("Primeras 5 filas de la tabla X final:")
display(X.head())

✅ 2. Características después de la Codificación (Protocolos convertidos).
Primeras 5 filas de la tabla X final:


,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.checksum,...,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,ip.proto_1,ip.proto_6,ip.proto_17
0,"Jan 14, 2025 18:40:10.397697000 GMT",68,eth:ethertype:ip:tcp:mqtt,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,0xbeed,...,502.0,-1.0,0x1fd0,0101080a6be0130a27064a75,2.0,NaN,NaN,False,True,False
1,"Jan 14, 2025 18:40:10.405220000 GMT",68,eth:ethertype:ip:tcp:mqtt,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.4.2,0x02,61,0x04c4,...,501.0,-1.0,0xae0f,0101080a2706ac346be0130a,2.0,NaN,NaN,False,True,False
2,"Jan 14, 2025 18:40:10.405272000 GMT",66,eth:ethertype:ip:tcp,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.4.2,192.168.20.31,0x02,64,0xbeee,...,502.0,-1.0,0x7e10,0101080a6be013112706ac34,NaN,NaN,NaN,False,True,False
3,"Jan 14, 2025 18:40:40.388553000 GMT",67,eth:ethertype:ip:udp:dns,02:42:e8:8a:b5:00,0c:03:36:87:00:00,192.168.0.2,192.168.20.31,0x02,64,0xde27,...,NaN,NaN,NaN,NaN,NaN,40294.0,53.0,False,False,True
4,"Jan 14, 2025 18:40:40.405604000 GMT",83,eth:ethertype:ip:udp:dns,0c:03:36:87:00:00,02:42:e8:8a:b5:00,192.168.20.31,192.168.0.2,0x02,61,0x7727,...,NaN,NaN,NaN,NaN,NaN,53.0,40294.0,False,False,True


**Paso 4: Preparación para el Modelo (División y Escalado:**

Necesitamos separar los datos para que el modelo aprenda y luego se pruebe de forma justa.

In [62]:
# BITÁCORA: PASO 4 - División y Escalado de Datos

# 4.1 Eliminación de Columnas No Numéricas
# Eliminar columnas que no son numéricas y no fueron codificadas, ya que no son directamente utilizables por el modelo.
# Basado en la exploración previa, estas columnas incluyen información de tiempo, direcciones MAC/IP, flags, checksums, y opciones.

columns_to_drop = ['frame.time', 'eth.src', 'eth.dst', 'ip.dst', 'ip.src', 'ip.flags', 'ip.checksum', 'ip.tos', 'tcp.options', 'tcp.pdu.size', 'udp.srcport', 'udp.dstport', 'tcp.checksum', 'tcp.flags']

# Asegurarse de que las columnas a eliminar existan en X_encoded antes de eliminarlas
columns_to_drop_existing = [col for col in columns_to_drop if col in X_encoded.columns]

X_processed = X_encoded.drop(columns=columns_to_drop_existing, axis=1)


# 4.2 División en Conjunto de Entrenamiento y Prueba
# Dividimos los datos procesados en un conjunto para entrenar el modelo (80%) y otro para probarlo (20%).
# random_state asegura que la división sea la misma cada vez que ejecutas el código.
# stratify=y asegura que la proporción de clases en y se mantenga en los conjuntos de entrenamiento y prueba.

X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42, stratify=y)

print("✅ Datos divididos en conjuntos de entrenamiento y prueba (estratificado).")
print(f"Tamaño del conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Tamaño del conjunto de prueba (X_test): {X_test.shape}")


# 4.3 Identificación de Columnas Numéricas para Escalado
# El escalado solo se aplica a columnas numéricas. Identificamos cuáles son.
# Ahora, después de eliminar las no numéricas, todas las columnas restantes deberían ser numéricas o binarias (de one-hot encoding).

numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()


# 4.4 Escalado de Características Numéricas
# Usamos StandardScaler para que las características numéricas tengan media 0 y desviación estándar 1.
# Esto ayuda a que muchos algoritmos de ML funcionen mejor.

scaler = StandardScaler()

# Aplicar el escalado SÓLO a las columnas numéricas del conjunto de entrenamiento
# Se manejan los NaNs que puedan existir en las columnas numéricas antes de escalar
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols].fillna(X_train[numeric_cols].mean()))

# Aplicar el escalado a las mismas columnas numéricas del conjunto de prueba
# Usamos transform() aquí, no fit_transform(), para usar los mismos parámetros aprendidos del conjunto de entrenamiento
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols].fillna(X_test[numeric_cols].mean()))


print("✅ Características numéricas escaladas con éxito.")
print("\nPrimeras 5 filas de X_train (escalado):")
display(X_train.head())
print("\nPrimeras 5 filas de X_test (escalado):")
display(X_test.head())

✅ Datos divididos en conjuntos de entrenamiento y prueba (estratificado).
Tamaño del conjunto de entrenamiento (X_train): (2000, 11)
Tamaño del conjunto de prueba (X_test): (501, 11)
✅ Características numéricas escaladas con éxito.

Primeras 5 filas de X_train (escalado):


,frame.len,ip.ttl,ip.proto,tcp.srcport,tcp.dstport,tcp.window_size_value,tcp.window_size_scalefactor,frame.protocols_eth:ethertype:ip:tcp,frame.protocols_eth:ethertype:ip:tcp:mqtt,frame.protocols_eth:ethertype:ip:udp:dns,frame.protocols_eth:ethertype:ip:udp:ntp
2081,1.655494,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False
2136,1.655494,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False
970,-0.647783,-1.304877,-0.525972,-1.624851,1.624851,1.314385,0.0,False,True,False,False
924,1.655494,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False
1910,1.655494,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False



Primeras 5 filas de X_test (escalado):


,frame.len,ip.ttl,ip.proto,tcp.srcport,tcp.dstport,tcp.window_size_value,tcp.window_size_scalefactor,frame.protocols_eth:ethertype:ip:tcp,frame.protocols_eth:ethertype:ip:tcp:mqtt,frame.protocols_eth:ethertype:ip:udp:dns,frame.protocols_eth:ethertype:ip:udp:ntp
2391,-0.650969,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False
1296,-0.612740,-1.304877,1.844726,-0.017451,0.017451,0.037583,0.0,False,False,False,True
1931,-0.650969,-1.304877,-0.525972,-1.624851,1.624851,2.307118,0.0,True,False,False,False
64,-0.647783,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,False,True,False,False
2258,1.655494,0.766356,-0.525972,0.802924,-0.802924,-0.670024,0.0,True,False,False,False


**Paso 5: Modelado Predictivo (Entrenamiento del IDS):**

Aquí ejecutamos el Algoritmo de Post-procesamiento central de la Fase 4.


**5.1 Código: Entrenamiento del Random Forest**

In [63]:
# BITÁCORA: PASO 5.1 - Definición y Entrenamiento del Modelo (Random Forest Classifier)
# Random Forest es un algoritmo de IA robusto para clasificación en seguridad IoT.
# class_weight='balanced' le da más peso a los ataques (clase minoritaria).
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# **TEMPORARY FIX:** Drop 'tcp.flags' if it still exists in X_train and X_test
# It's recommended to handle this in the data processing step (Paso 4).
if 'tcp.flags' in X_train.columns:
    X_train = X_train.drop('tcp.flags', axis=1)
if 'tcp.flags' in X_test.columns:
    X_test = X_test.drop('tcp.flags', axis=1)


# Entrenamiento: El modelo aprende de los datos escalados (X_train).
print("Iniciando entrenamiento del modelo...")
model.fit(X_train, y_train)
print("✅ Modelo entrenado con éxito.")

# El modelo 'model' ahora está listo para hacer predicciones.

Iniciando entrenamiento del modelo...
✅ Modelo entrenado con éxito.


In [64]:
# Check the class distribution in the test set
print("Conteo de valores en el conjunto de prueba (y_test):")
display(y_test.value_counts())

Conteo de valores en el conjunto de prueba (y_test):


,count
label,
Benign,501


**Paso 6: Evaluación y Post-procesamiento:**

Medimos qué tan bien predijo el modelo los ataques que estaban en el conjunto de prueba.

In [65]:
# BITÁCORA: PASO 6 - Evaluación y Post-procesamiento

# 6.1 Predicción en el Conjunto de Prueba
# Usamos el modelo entrenado para hacer predicciones sobre los datos que nunca antes ha visto (el conjunto de prueba).

y_pred = model.predict(X_test)

print("✅ Predicciones realizadas en el conjunto de prueba.")


# 6.2 Evaluación del Modelo
# Comparamos las predicciones del modelo (y_pred) con las etiquetas reales del conjunto de prueba (y_test).
# classification_report nos da métricas clave como precisión, recall, f1-score y soporte para cada clase.

print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred))

# Ahora puedes interpretar los resultados para entender el rendimiento del modelo.

✅ Predicciones realizadas en el conjunto de prueba.

Informe de Clasificación:
              precision    recall  f1-score   support

      Benign       1.00      1.00      1.00       501

    accuracy                           1.00       501
   macro avg       1.00      1.00      1.00       501
weighted avg       1.00      1.00      1.00       501

